Calculate steric SSH changes in the region to plot maps as 

$$
-\int \frac{\rho - \rho_{0}}{\rho_{0}} dz
$$

with $\rho_{0} = 1035$. Both density and cell height are taken from the same experiment. Subsequently, divide into thermosteric and halosteric contributions. Calculate the bottom pressure term as well, following Griffies et al. (2014). 

In [1]:
import cmocean 
import cosima_cookbook as cc
import cosima_cookbook.distributed as ccd
import dask.distributed as dsk
import gsw
import numpy as np
import xarray as xr

import warnings # ignore these warnings
warnings.filterwarnings("ignore", category = FutureWarning)
warnings.filterwarnings("ignore", category = UserWarning)
warnings.filterwarnings("ignore", category = RuntimeWarning)

figdir = '/home/561/jn8053/g_e14/figures-tmp/easterlies-collaborative/'

clnt = dsk.Client()
clnt

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 4
Total threads: 16,Total memory: 250.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:45201,Workers: 4
Dashboard: /proxy/8787/status,Total threads: 16
Started: Just now,Total memory: 250.00 GiB
Comm: tcp://127.0.0.1:35197,Total threads: 4
Dashboard: /proxy/33195/status,Memory: 62.50 GiB
Nanny: tcp://127.0.0.1:43107,


In [2]:
session = cc.database.create_session()
experiment_ryf = '01deg_jra55v13_ryf9091'
experiment_eup = '01deg_jra55v13_ryf9091_easterlies_up10'
experiment_edo = '01deg_jra55v13_ryf9091_easterlies_down10'

ti = '2150-01-01'
tf = '2160-12-31'

In [3]:
ht = cc.querying.getvar(experiment_ryf, 'ht', session, n = 1).sel(yt_ocean = slice(None, -58))

In [4]:
# Height of t-cells
dzt = {}
dzt['ctrl'] = cc.querying.getvar(experiment_ryf, 'dzt', session, frequency = '1 monthly', start_time = ti, end_time = tf).sel(time = slice(ti, tf), yt_ocean = slice(None, -50))
dzt['up'] = cc.querying.getvar(experiment_eup, 'dzt', session, frequency = '1 monthly', start_time = ti, end_time = tf).sel(time = slice(ti, tf), yt_ocean = slice(None, -50))
dzt['down'] = cc.querying.getvar(experiment_edo, 'dzt', session, frequency = '1 monthly', start_time = ti, end_time = tf).sel(time = slice(ti, tf), yt_ocean = slice(None, -50))

In [5]:
# Temperature and salinity
temp = {}
salt = {}
temp['ctrl'] = cc.querying.getvar(experiment_ryf, 'temp', session, frequency = '1 monthly', start_time = ti, end_time = tf).sel(time = slice(ti, tf), yt_ocean = slice(None, -58))-273.15
temp['up'] = cc.querying.getvar(experiment_eup, 'temp', session, frequency = '1 monthly', start_time = ti, end_time = tf).sel(time = slice(ti, tf), yt_ocean = slice(None, -58))-273.15
temp['down'] = cc.querying.getvar(experiment_edo, 'temp', session, frequency = '1 monthly', start_time = ti, end_time = tf).sel(time = slice(ti, tf), yt_ocean = slice(None, -58))-273.15
salt['ctrl'] = cc.querying.getvar(experiment_ryf, 'salt', session, frequency = '1 monthly', start_time = ti, end_time = tf).sel(time = slice(ti, tf), yt_ocean = slice(None, -58))
salt['up'] = cc.querying.getvar(experiment_eup, 'salt', session, frequency = '1 monthly', start_time = ti, end_time = tf).sel(time = slice(ti, tf), yt_ocean = slice(None, -58))
salt['down'] = cc.querying.getvar(experiment_edo, 'salt', session, frequency = '1 monthly', start_time = ti, end_time = tf).sel(time = slice(ti, tf), yt_ocean = slice(None, -58))

2022-08-19 09:41:21,737 - distributed.utils_perf - WARNING - full garbage collections took 25% CPU time recently (threshold: 10%)
2022-08-19 09:41:22,034 - distributed.utils_perf - WARNING - full garbage collections took 28% CPU time recently (threshold: 10%)
2022-08-19 09:41:22,099 - distributed.utils_perf - WARNING - full garbage collections took 27% CPU time recently (threshold: 10%)


In [6]:
rho_0 = 1035

# Total steric ssh

In [13]:
for k in ['ctrl', 'up', 'down']:
    temp_bott = temp[k]
    salt_bott = salt[k]

    p = gsw.p_from_z(-temp_bott['st_ocean'], temp_bott['yt_ocean'])
    salt_abs = gsw.SA_from_SP(salt_bott, 
                              p, 
                              salt_bott['xt_ocean'], 
                              salt_bott['yt_ocean'])

    # Density changes due to temp/salt changes
    rho_anom = (gsw.rho(salt_bott, temp_bott, p) - rho_0) / rho_0

    # Integrating surface layer
    steric_ssh = (rho_anom * dzt[k]).sum('st_ocean')
    steric_ssh = steric_ssh.chunk({'time':'500MB'})
    steric_ssh = ccd.compute_by_block(steric_ssh)
    steric_ssh.rename('steric_ssh').to_netcdf('steric_ssh_'+k+'.nc')

  0%|          | 0/132 [00:00<?, ?it/s]

  0%|          | 0/132 [00:00<?, ?it/s]

## Thermosteric contribution

In [8]:
for k in ['ctrl', 'up', 'down']:
    temp_bott = temp[k]
    salt_bott = salt[k].isel(time = slice(None, 12)).mean('time')

    p = gsw.p_from_z(-temp_bott['st_ocean'], temp_bott['yt_ocean'])
    salt_abs = gsw.SA_from_SP(salt_bott, p, 
                              salt_bott['xt_ocean'], 
                              salt_bott['yt_ocean'])

    # Density changes due to temp/salt changes
    rho_anom = (gsw.rho(salt_bott, temp_bott, p) - rho_0) / rho_0

    # Integrating surface layer
    steric_ssh = (rho_anom * dzt[k]).sum('st_ocean')
    steric_ssh = steric_ssh.chunk({'time':'500MB'})
    steric_ssh = ccd.compute_by_block(steric_ssh)
    steric_ssh.rename('thermosteric_ssh').to_netcdf('thermosteric_ssh_'+k+'.nc')

  0%|          | 0/18 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

## Halosteric contribution

In [7]:
for k in ['ctrl', 'up', 'down']:
    temp_bott = temp[k].isel(time = slice(None, 12)).mean('time')
    salt_bott = salt[k]

    p = gsw.p_from_z(-temp_bott['st_ocean'], temp_bott['yt_ocean'])
    salt_abs = gsw.SA_from_SP(salt_bott, p, 
                              salt_bott['xt_ocean'], 
                              salt_bott['yt_ocean'])

    # Density changes due to temp/salt changes
    rho_anom = (gsw.rho(salt_bott, temp_bott, p) - rho_0) / rho_0

    # Integrating surface layer
    steric_ssh = (rho_anom * dzt[k]).sum('st_ocean')
    steric_ssh = steric_ssh.chunk({'time':'500MB'})
    steric_ssh = ccd.compute_by_block(steric_ssh)
    steric_ssh.rename('halosteric_ssh').to_netcdf('halosteric_ssh_'+k+'.nc')

  0%|          | 0/18 [00:00<?, ?it/s]

2022-08-19 09:41:53,549 - distributed.utils_perf - WARNING - full garbage collections took 22% CPU time recently (threshold: 10%)
2022-08-19 09:42:09,100 - distributed.utils_perf - WARNING - full garbage collections took 27% CPU time recently (threshold: 10%)
2022-08-19 09:42:19,051 - distributed.utils_perf - WARNING - full garbage collections took 16% CPU time recently (threshold: 10%)
2022-08-19 09:42:31,147 - distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
2022-08-19 09:42:31,811 - distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
2022-08-19 09:42:39,404 - distributed.utils_perf - WARNING - full garbage collections took 12% CPU time recently (threshold: 10%)
2022-08-19 09:43:01,910 - distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
2022-08-19 09:43:02,140 - distributed.utils_perf - WARNING - full garbage collections took

  0%|          | 0/80 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

# Mass term

In [10]:
pbot = {}
pbot['ctrl'] = cc.querying.getvar(experiment_ryf, 'pbot_t', session, 
                                  frequency = '1 monthly', start_time = ti, 
                                  end_time = tf).sel(time = slice(ti, tf), 
                                                     yt_ocean = slice(None, -58))
pbot['up'] = cc.querying.getvar(experiment_eup, 'pbot_t', session, 
                                frequency = '1 monthly', start_time = ti, 
                                end_time = tf).sel(time = slice(ti, tf), 
                                                   yt_ocean = slice(None, -58))
pbot['down'] = cc.querying.getvar(experiment_edo, 'pbot_t', session, 
                                  frequency = '1 monthly', start_time = ti, 
                                  end_time = tf).sel(time = slice(ti, tf), 
                                                     yt_ocean = slice(None, -58))

In [14]:
# Create a shelf mask
ht = cc.querying.getvar(experiment_ryf, 'ht', session, n = 1).sel(yt_ocean = slice(None, -58))

iso = xr.open_dataset('~/g_e14/misc/Antarctic_isobath_1000m.nc')
lat_slice = slice(-80,-58)
lon_slice = slice(-280, 80)

def shelf_mask_isobath(var):
    '''
    Masks ACCESS-OM2-01 variables by the region polewards of the 1000m isobath as computed using 
    a script contributed by Adele Morrison.
    Only to be used with ACCESS-OM2-0.1 output!
    '''
    contour_file = np.load('/g/data/ik11/grids/Antarctic_slope_contour_1000m.npz')
    shelf_mask = contour_file['contour_masked_above']
    yt_ocean = contour_file['yt_ocean']
    xt_ocean = contour_file['xt_ocean']
    # Mask values that are non-zero
    shelf_mask[np.where(shelf_mask!=0)] = np.nan
    shelf_mask = shelf_mask+1
    shelf_map = np.nan_to_num(shelf_mask)
    shelf_mask = xr.DataArray(shelf_mask, coords = [('yt_ocean', yt_ocean), ('xt_ocean', xt_ocean)])
    shelf_map = xr.DataArray(shelf_map, coords = [('yt_ocean', yt_ocean), ('xt_ocean', xt_ocean)])
    # Multiply the variable with the mask, we need to account for the shape of the mask. 
    # The mask uses a northern cutoff of 59S.
    masked_var = var.sel(yt_ocean = lat_slice) * shelf_mask
    return masked_var, shelf_map

ht_masked, shelf_mask = shelf_mask_isobath(ht)

In [15]:
for k in ['ctrl', 'up', 'down']:
    pbot_shelf, shelf_mask = shelf_mask_isobath(pbot[k])
    pbot_shelf = (1/(9.8*1035))*(pbot_shelf*1e4).mean(['xt_ocean', 'yt_ocean'])
    pbot_shelf = pbot_shelf.load()

    pbot_shelf.rename('mass_term').to_netcdf('mass_term_shelf_ave_'+k+'.nc')